# Data Mining Project Assignment

## Importing Libraries

In [3]:
%matplotlib inline
import gradio as gr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import feyn
from sklearn.model_selection import train_test_split
import xgboost as xg 
from sklearn.svm import SVR 
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV, Lasso
from sklearn.preprocessing import StandardScaler

## Feature Engineering

In [4]:
data = pd.read_csv('/Users/haroonshoaib/Developer/Github/EnergyConsumptionPrediction/EnergyConsumptionPredictionFiles/clean-preprocessed_last.csv')
data.rename(columns = {'Unnamed: 0':'date'}, inplace = True)
data = data.drop(['year'],axis=1)
#data = data.astype({'week_of_year':'string'})
data = data.astype({'sun_rise':'int'})
data['date'] = pd.to_datetime(data['date'])
data['year'] = data['date'].dt.year
data['day'] = data['date'].dt.day
data['month'] = data['date'].dt.month
data = data.drop(['date','sun_rise'],axis=1)

#print(data)
#gr.themes.builder()

## Train-test Split

In [5]:
x = data.loc[:, data.columns != 'hourly_demand'].values
y = data.loc[:, 'hourly_demand'].values
x_graph = data.loc[9100:, data.columns != 'hourly_demand'].values
pred_graph = data.loc[9100:, 'hourly_demand'].values
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
print(x_graph.shape)
print(x.shape)
print(pred_graph.shape)
qtrain, qtest = feyn.tools.split(data, ratio=[0.7, 0.3], random_state=42)
qytest = qtest.loc[:, 'hourly_demand'].values
qytest
#y = data.loc[:, 'hourly_demand'].values

(31, 42)
(9131, 42)
(31,)


array([13342., 14272., 14744., ..., 16863., 13590., 13887.])

In [6]:
# Instantiate a QLattice
ql = feyn.QLattice(random_seed=42)

# Sample and fit models
qmodels = ql.auto_run(
    data=qtrain,
    output_name='hourly_demand'
)
best = qmodels[0]

## Training Data with XGBoost Model

In [7]:
xgb_r = xg.XGBRegressor(objective ='reg:squarederror', n_estimators = 10, seed = 123)
xgb_r.fit(x_train, y_train) 

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=10, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

## Training Data with SVR Model

In [8]:
svr = SVR(kernel='linear') 
svr.fit(x_train, y_train) 

SVR(kernel='linear')

## Training Data with Random Forest Regressor Model

In [9]:
rf = RandomForestRegressor(n_estimators=100,random_state=0)
rf.fit(x_train,y_train)

RandomForestRegressor(random_state=0)

## Training Data with Linear Regression Model

In [10]:
model = LinearRegression()
model.fit(x_train, y_train)

LinearRegression()

## Scaling data and training the data with Lasso Regression model

In [11]:
sc_X = StandardScaler()
X_train = sc_X.fit_transform(x_train)
X_test = sc_X.fit_transform(x_test)
model = LassoCV(cv=10, random_state=42, max_iter=50000)
model.fit(X_train, y_train)
lasso_best = Lasso(alpha=model.alpha_)
lasso_best.fit(X_train, y_train)
#Y_pred = lasso_best.predict(X_test)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.471e+09, tolerance: 1.861e+06
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=0.7995701914430793)

In [12]:
p_xgb = xgb_r.predict(x_test)
p_rf = rf.predict(x_test)
p_svr = svr.predict(x_test)
p_mlr = model.predict(X_test)
p_lr = lasso_best.predict(X_test)
p_ql = best.predict(qtest)

p_ql

array([12881.02733162, 13941.54180306, 14869.97034281, ...,
       17796.37591006, 13780.72231352, 14552.31557313])

In [13]:
#r_xgb = metrics.r2_score(y_test,p_xgb)
#r_rf = metrics.r2_score(y_test,p_rf)
#r_svr = metrics.r2_score(y_test,p_svr)

In [14]:
#srs = data.loc[(data['month'] == 3) & (data['year'] == 2009),data.columns != 'hourly_demand' ].values
#act_val = data.loc[(data['month'] == 3) & (data['year'] == 2009),data.columns == 'hourly_demand' ].values
#y_graph = xgb_r.predict(srs)
#x_fin = data.loc[(data['month'] == 3) & (data['year'] == 2009),data.columns == 'day' ].values
#print(x_fin)
#print(act_val.shape)
#print(y_graph.shape)
#print(srs.shape)
#plt.plot(x_fin,y_graph,label = 'Predictions for Selected Month')
#plt.plot(x_fin,act_val,label = 'Actual values for Selected Month')

In [15]:
def make_plot(m,y):
    #srs = data[(data['month'] == m) & (df['year'] == y)]
    srs = data.loc[(data['month'] == m) & (data['year'] == y),data.columns != 'hourly_demand' ].values
    act_val = data.loc[(data['month'] == m) & (data['year'] == y),data.columns == 'hourly_demand' ].values
    x_fin = data.loc[(data['month'] == m) & (data['year'] == y),data.columns == 'day' ].values
    y_graph = xgb_r.predict(srs) 
    plt.plot(x_fin,y_graph,label = 'Predictions for Selected Month')
    plt.plot(x_fin,act_val,label = 'Actual values for Selected Month')
    plt.legend(loc="upper left")
    plt.xlabel('DAY')
    plt.ylabel('Energy Consumption (MWh)')
    #plt.show()
    #demo.load(make_plot,inputs=[graph_button], outputs=[plot])
    return plt
def calc_xgb(temp_min, temp_max, temp_mean, temp_median, temp_max_hour, temp_min_hour, dew_point_temp_min, dew_point_temp_max,
             dew_point_temp_mean,
             dew_point_temp_median, dew_point_temp_max_hour, dew_point_temp_min_hour, rel_hum_min, rel_hum_max, rel_hum_mean, rel_hum_median,
             rel_hum_max_hour,
             rel_hum_min_hour, visibility_min, visibility_max, visibility_mean, visibility_median, visibility_max_hour, 
             visibility_min_hour,day_of_week,week_of_year,day,month,year,
             press_min, press_max, press_mean, press_median, press_max_hour, press_min_hour, 
             hmdxx_min, hmdxx_max, hmdxx_mean,hmdxx_median, hmdxx_max_hour, hmdxx_min_hour,sun_set):
    x_arr = [temp_min, temp_max, temp_mean, temp_median, temp_max_hour, temp_min_hour, dew_point_temp_min, dew_point_temp_max,
             dew_point_temp_mean,
             dew_point_temp_median, dew_point_temp_max_hour, dew_point_temp_min_hour, rel_hum_min, rel_hum_max, rel_hum_mean, rel_hum_median,
             rel_hum_max_hour,
             rel_hum_min_hour, visibility_min, visibility_max, visibility_mean, visibility_median, visibility_max_hour, 
             visibility_min_hour,day_of_week,week_of_year,day,month,year,
             press_min, press_max, press_mean, press_median, press_max_hour, press_min_hour, 
             hmdxx_min, hmdxx_max, hmdxx_mean,hmdxx_median, hmdxx_max_hour, hmdxx_min_hour,sun_set]
    pred = xgb_r.predict(np.array(x_arr).reshape(1,42))

    return pred[0]
def r_square():
    r_xgb = metrics.r2_score(y_test,p_xgb)
    r_rf = metrics.r2_score(y_test,p_rf)
    r_svr = metrics.r2_score(y_test,p_svr)
    r_mlr = metrics.r2_score(y_test,p_mlr)
    r_lr = metrics.r2_score(y_test,p_lr)
    r_ql = metrics.r2_score(qytest,p_ql)

    mae_xgb = metrics.mean_absolute_error(y_test, p_xgb)
    mae_rf = metrics.mean_absolute_error(y_test, p_rf)
    mae_svr = metrics.mean_absolute_error(y_test, p_svr)
    mae_mlr = metrics.mean_absolute_error(y_test, p_mlr)
    mae_lr = metrics.mean_absolute_error(y_test, p_lr)
    mae_ql = metrics.mean_absolute_error(qytest, p_ql)

    mse_xgb = metrics.mean_squared_error(y_test, p_xgb)
    mse_rf = metrics.mean_squared_error(y_test, p_rf)
    mse_svr = metrics.mean_squared_error(y_test, p_svr)
    mse_mlr = metrics.mean_squared_error(y_test, p_mlr)
    mse_lr = metrics.mean_squared_error(y_test, p_lr)
    mse_ql = metrics.mean_squared_error(qytest, p_ql)

    rmse_xgb = metrics.mean_squared_error(y_test, p_xgb, squared=False)
    rmse_rf = metrics.mean_squared_error(y_test, p_rf, squared=False)
    rmse_svr = metrics.mean_squared_error(y_test, p_svr, squared=False)
    rmse_mlr = metrics.mean_squared_error(y_test, p_mlr, squared=False)
    rmse_lr = metrics.mean_squared_error(y_test, p_lr, squared=False)
    rmse_ql = metrics.mean_squared_error(qytest, p_ql, squared=False)
    arraytype = [r_xgb,r_rf,r_svr,r_mlr,r_lr,r_ql,mae_xgb,mae_rf,mae_svr,mae_mlr,mae_lr,mae_ql,mse_xgb,mse_rf,
                 mse_svr,mse_mlr,mse_lr,mse_ql,rmse_xgb,rmse_rf,rmse_svr,rmse_mlr,rmse_lr,rmse_ql]
    #np.array()
    return arraytype

## GUI

In [16]:
with gr.Blocks() as demo:
    gr.Markdown("PREDICT ELECTRICITY CONSUMPTION IN HOUSEHOLD")
    with gr.Tab("XGBoost"):
        with gr.Row():
            temp_min=gr.Number(label="Temp Min (in Celsius)" )
            temp_max=gr.Number(label="Temp Max (in Celsius)" )
            temp_mean=gr.Number(label="Temp Mean (in Celsius)" )
            temp_median=gr.Number(label="Temp Median (in Celsius)")
        with gr.Row():
            temp_min_hour=gr.Number(label="Temp Min Hour" )
            temp_max_hour=gr.Number(label="Temp Max Hour")
        
        with gr.Row():
            dew_point_temp_min=gr.Number(label="Dew Point Temp Min (in Celsius)")
            dew_point_temp_max=gr.Number(label="Dew Point Temp Max (in Celsius)")
            dew_point_temp_mean=gr.Number(label="Dew Point Temp Mean (in Celsius)")
            dew_point_temp_median=gr.Number(label="Dew Point Temp Median (in Celsius)")
        with gr.Row():
            dew_point_temp_min_hour=gr.Slider(0,24,label="Dew Point Temp Min Hour", info="Choose between 0 and 24")
            dew_point_temp_max_hour=gr.Slider(0,24,label="Dew Point Temp Max Hour", info="Choose between 0 and 24")

        with gr.Row():
            rel_hum_min = gr.Number(label="Rel Hum Min (in percentage)")
            rel_hum_max = gr.Number(label="Rel Hum Max (in percentage)")
            rel_hum_mean = gr.Number(label="Rel Hum Mean (in percentage)")
            rel_hum_median = gr.Number(label="Rel Hum Median (in percentage)")

        with gr.Row():
            rel_hum_max_hour = gr.Number(label="Rel Hum Max Hour", info="Choose between 0 and 24")
            rel_hum_min_hour = gr.Number(label="Rel Hum Min Hour", info="Choose between 0 and 24")

        with gr.Row():
            visibility_min = gr.Slider(0,60,label="Visibility Min (in kms)", info="Choose between 0 and 60")
            visibility_max = gr.Slider(0,60,label="Visibility Max (in kms)", info="Choose between 0 and 60")
            visibility_mean = gr.Slider(0,60,label="Visibility Mean (in kms)", info="Choose between 0 and 60")
            visibility_median = gr.Slider(0,60,label="Visibility Median (in kms)", info="Choose between 0 and 60")

        with gr.Row():
            visibility_max_hour = gr.Number(label="Visibility Max Hour", info="Choose between 0 and 24")
            visibility_min_hour = gr.Number(label="Visibility Min Hour", info="Choose between 0 and 24")

        with gr.Row():
            press_min=gr.Number(label="Pressure Min (in kPa)")
            press_max=gr.Number(label="Pressure Max (in kPa)")
            press_mean=gr.Number(label="Pressure Mean (in kPa)")
            press_median=gr.Number(label="Pressure Median (in kPa)")
        with gr.Row():
            press_max_hour=gr.Number(label="Pressure Max Hour", info="Choose between 0 and 24")
            press_min_hour=gr.Number(label="Pressure Min Hour", info="Choose between 0 and 24")
        with gr.Row():
            hmdxx_min=gr.Number(label="Humidex Min")
            hmdxx_max=gr.Number(label="Humidex Max")
            hmdxx_mean=gr.Number(label="Humidex Mean")
            hmdxx_median=gr.Number(label="Humidex Median")
        
        with gr.Row():
            hmdxx_max_hour=gr.Number(label="Humidex Max Hour", info="Choose between 0 and 24")
            hmdxx_min_hour=gr.Number(label="Humidex Min Hour", info="Choose between 0 and 24")
            day_of_week = gr.Number(label="Day Of Week", info="Choose between 0 and 7")
        with gr.Row():
            day=gr.Number(label="Day", info="Choose between 1 and 31")
            week_of_year = gr.Number(label="Week Of Year", info="Choose between 1 and 52")
            month=gr.Number(label="Month", info="Choose between 1 and 12")
            year=gr.Number(label="Year")
        with gr.Row():
            sun_set=gr.Slider(0, 24, label="Sun Set", info="Choose between 0 and 24")
        
        text_output = gr.Textbox(label='Predicted Value (in MWh)')
        
        xgb_button = gr.Button("Predict")
    with gr.Tab("Graph"):
        #dim=gr.Number(label="Hmdxx Min", info="Choose between -50 and 50")
        
        m=gr.Number(label="Month", info="Choose between -50 and 50")
        y=gr.Number(label="Year", info="Choose between -50 and 50")
        plot = gr.Plot(label="Plot")
        graph_button = gr.Button("Display Graph")
        #plot = gr.Plot(label="Plot")
        graph_button.click(make_plot,inputs = [m,y],outputs=[plot])
        #demo.load(make_plot,outputs=[plot])
        
    with gr.Tab("Evaluation Metrics"):
        with gr.Row():
            gr.Label('R-Squared Values')
            xgb_output = gr.Textbox(label='XGB')
            rf_output = gr.Textbox(label='RF')
            svr_output = gr.Textbox(label='SVR')
            mlr_output = gr.Textbox(label='MLR')
            lr_output = gr.Textbox(label='LR')
            ql_output = gr.Textbox(label='QLattice')
        with gr.Row():
            gr.Label('Mean Absolute Error Values')
            a = gr.Textbox(label='XGB')
            b = gr.Textbox(label='RF')
            c=gr.Textbox(label='SVR')
            d=gr.Textbox(label='MLR')
            e=gr.Textbox(label='LR')
            f = gr.Textbox(label='QLattice')
        with gr.Row():
            gr.Label('Mean Squared Error Values')
            aa=gr.Textbox(label='XGB')
            bb=gr.Textbox(label='RF')
            cc=gr.Textbox(label='SVR')
            dd=gr.Textbox(label='MLR')
            ee=gr.Textbox(label='LR')
            ff = gr.Textbox(label='QLattice')
        with gr.Row():
            gr.Label('Root Mean Squared Error Values')
            aaa=gr.Textbox(label='XGB')
            bbb=gr.Textbox(label='RF')
            ccc=gr.Textbox(label='SVR')
            ddd=gr.Textbox(label='MLR')
            eee=gr.Textbox(label='LR')
            fff = gr.Textbox(label='QLattice')
            

        disp_button = gr.Button("Display")
    
    xgb_button.click(calc_xgb, inputs=[temp_min, temp_max, temp_mean, temp_median, temp_max_hour, temp_min_hour, dew_point_temp_min, dew_point_temp_max,
             dew_point_temp_mean,
             dew_point_temp_median, dew_point_temp_max_hour, dew_point_temp_min_hour, rel_hum_min, rel_hum_max, rel_hum_mean, rel_hum_median,
             rel_hum_max_hour,
             rel_hum_min_hour, visibility_min, visibility_max, visibility_mean, visibility_median, visibility_max_hour, 
             visibility_min_hour,day_of_week,week_of_year,day,month,year,
             press_min, press_max, press_mean, press_median, press_max_hour, press_min_hour, 
             hmdxx_min, hmdxx_max, hmdxx_mean,hmdxx_median, hmdxx_max_hour, hmdxx_min_hour,sun_set], outputs=text_output)
    disp_button.click(r_square,  outputs=[xgb_output,rf_output,svr_output,mlr_output,lr_output,ql_output,a,b,c,d,e,f,aa,bb,cc,dd,ee,ff,aaa,bbb,ccc,ddd,eee,fff])

demo.launch()

Running on local URL:  http://127.0.0.1:7862

Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 4.7.1, however version 4.29.0 is available, please upgrade.
--------
